In [205]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
from intvalpy import lineqs
import sympy
import numpy as np
import os
import json
from scipy.optimize import linprog
from ipywidgets import interact, widgets
from functools import lru_cache

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \leq b\\
& && x \geq 0\\
& && x_i \in \mathbb{Z}, i \in I
\end{align}
$$

In [206]:
def run_cutting_planes():
    # Call Cutting Planes Binary
    cmd = './bin/bar --option --otheroption.jpg'
    b = os.popen(cmd).read()
    print(b)

def load_data():
    return json.load(open('test.json'))

d = load_data()
print(d)

# Access the system data
n, m = d['n'], d['m']
A = -np.array(d['A'], dtype=np.float64)
b = -np.array(d['b'], dtype=np.float64)
c = np.array(d['c'], dtype=np.float64)
I = np.array(d['I'], dtype=bool)
sols = d['sols']
n_cuts = b.shape[0] - m
assert(A.shape[0] == b.shape[0] and n == A.shape[1] and n == c.shape[0])
assert(n == 2)

{'A': [[3.0, 2.0], [-3.0, 2.0], [0.0, -1.0], [0.0, 1.0], [-1.0, 1.0]], 'I': [True, True], 'b': [6.0, 0.0, 0.0, 1.0, 0.0], 'c': [0.0, -1.0], 'm': 3, 'n': 2, 'sols': [[1.0, 1.5], [0.6666666666666666, 1.0], [1.0, 1.0]]}


In [207]:
import numpy as np
import matplotlib.pyplot as plt

def plot_lp(c, A, b, sol, xlim=(-.5, 2.5), ylim=(-.5, 2)):
    n = len(c)
    assert(n == 2)
    m = len(b)

    # Generate Meshgrid of Points
    x_vals = np.linspace(xlim[0], xlim[1], 400)
    y_vals = np.linspace(ylim[0], ylim[1], 400)
    X, Y = np.meshgrid(x_vals, y_vals)

    # Plot Objective Function
    Z = c[0]*X + c[1]*Y
    plt.contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.3)

    # Plot Inequality Constraints
    [plt.plot(x_vals, (b[i] - A[i,0]*x_vals) / A[i,1], 'g-', lw=2) for i in range(m)]

    # Shade Feasible Region
    feasible_cmap = matplotlib.colors.ListedColormap(['white', 'lime'])
    feasible_region = np.all(A[:,0,None,None]*X + A[:,1,None,None]*Y >= b[:,None,None], axis=0)
    plt.contourf(X, Y, feasible_region, cmap=feasible_cmap, alpha=0.2)

    # Plot Optimal Solution
    plt.plot(sol[0],sol[1],'ro')

    # Labels and Stuff
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Cutting Planes')
    plt.grid(color='gray', linestyle='-', linewidth=1)
    plt.xticks(np.arange(np.floor(xlim[0]), xlim[1], 1))
    plt.yticks(np.arange(np.floor(ylim[0]), ylim[1], 1))
    plt.xlim(xlim)
    plt.ylim(ylim)

    # Show plot
    plt.show()

In [208]:
def plot_cutting_planes(i):
    plot_lp(c, A[:m+i,:], b[:m+i], sols[i])

interact(plot_cutting_planes, i=widgets.IntSlider(min=0, max=n_cuts, step=1, value=0, description='Iteration:'))

interactive(children=(IntSlider(value=0, description='Iteration:', max=2), Output()), _dom_classes=('widget-in…

<function __main__.plot_cutting_planes(i)>